# 최종 프로젝트
- 2일차

- WBS 작성
- 요구기능 정의서 작성
- 인프라 보안 PDF 정리 및 프로젝트 진행 계획 간략하게 작성
- 네트워크 구성도 구상
- 서버 장비 확인
 

## 인프라 보안 PDF 정리
1. 대략적인 진행 순서
    1. 환경 구축
    2. 공격 시나리오 진행
    3. 대응
    4. 개선사항 확인
    5. MLTK 적용
    6. 정/오탐 확인 및 침해사고 대응 보고서 작성 
2. 환경 구축 관련
    - ESXI
        - ID : root  /  PW : rhkswp123!
        - IP : 172.25.0.129
        - Subnet Mask : 255.255.255.0
        - Gateway : 172.25.0.2
    - mint-client
        - IP : 192.168.233.129 (나중에 수정)
        - Subnet Mask : 255.255.255.0
    - ubuntu
        - ID : gasbugs  /   PW : rhkswp123
        - IP : 10.0.0.100/24
    - kali
        - ID/PW : kali
        - IP : 192.168.233.135(나중에 수정)
    


- SIEM: Splunk 사용. 연동된 장비에서 로그를 받아 통합 분석 및 관리
- 장비: 방화벽, IDS/IPS, 웹 방화벽
- 네트워크: 외부망/내부망/DMZ
- 서버: DB 서버, 관제 플랫폼 운영 서버(로그 수집, 분석, 저장, 관리), 관제 대상 시스템 서버(보호 대상)
- 가상환경: ESXI
- OS:
    - 클라이언트용: 리눅스 민트 (RAM 1GB, DISC 16GB)
    - 서버용: Ubuntu 20.04(DISC 16G, openssh 서버 설치)
    - 공격용: Kali linux
- 가상 스위치를 사용한 네트워크 분리
    - 인트라넷: Splunk, IDS, DB —> 무차별 모드 설정 네트워크
    - 외부 인터넷: 공격자/인터넷
    - DMZ: WAF, LB, 웹 서비스
    - 인트라넷-내부 직원: 정보보호본부(soc-net), IT개발본부(dev-net)
- VyOS 라우터
    - 소프트웨어 기반 네트워크 라우팅, 방화벽 및 VPN 기능을 제공하는 리눅스 기반 네트워크 운영 체제
    - VyOS ova 1.1.8버전 파일을 ESXI에 배포
- ESXI 유저 관리
    - 사용자 추가
    - 역할 추가
    - 사용자에 룰 할당 → 가상 시스템의 사용권한 탭 → 사용권한 → 사용자 추가
- 방화벽
    - pfsense 다운 및 업로드
    - 새로운 가상머신 생성: CPU 1, RAM 1GB
    - 설치 후 GUI 인터페이스 접속: 민트 운영체제를 soc-net으로 연결하고 IP를 10.0.100.2로 설정하여 정보보호본부 네트워크 대역에 배치
    - pfsense 위자드 설정
- DMZ 구간 웹 서비스 구축
    - beebox와 bWAPP —> 웹 애플리케이션 취약점 시나리오 실습
        - beebox 설치 및 가상머신 생성 → 비박스 네트워크 인터페이스 설정하기
    - pfsense와 웹 서비스 통신 설정하기
    - 방화벽 —> 웹 서비스 포트포워딩 설정(firewall → NAT → Port Forwad)
- pfsense에 추가 패키지 설치
    - squid 설치 및 설정
- IDS/IPS 구축
    - pfsense 활용 IPS 구축
    - Security Onion 활용 IDS 구축(Suricata)
    - 관리자 PC로부터 IDS 장비로 연결할 수 있도록 방화벽 룰 설정
    - 패턴 탐지를 위한 Snort 룰 작성
- 웹 방화벽 구축
    - VLAN을 활용한 망분리
    - VLAN 인터페이스 구성 → 두 인터페이스에 각각 IP와 대역대 할당 → DHCP 서비스 생성 → 각 인터페이스에 방화벽 룰을 사용해 ACL 설정 → VLAN 포트 할당
    - soc-net, dev-mint를 활용하여 VLAN 환경 테스트

운영체제 (우분투)

도커 까는법

https://lordofkangs.tistory.com/279

도커에 mysql 까는법 

https://steemit.com/upvu/@realmankwon/ubuntu-docker-mysql

테이블은 chatgpt로 기호에 맞춰서 진행 해주시면 될거같아요

비박스  도커 연결

# 1) 네트워크 확인 (Ubuntu ↔ bee-box 서로 통신 가능한지)

Ubuntu(도커 올린 쪽)에서 IP 확인:

```bash
ip addr show
# 예: 192.168.56.101 같은 주소 찾기 (host-only) 또는 192.168.0.10 (bridge)

```

bee-box에서 IP 확인:

```bash
ip addr show
# 또는 ifconfig (설치되어 있으면)

```

Ubuntu에서 bee-box로 ping 해보기:

```bash
ping <BEEBOX_IP>

```

정상적으로 응답하면 네트워크 연결은 OK.

> 만약 서로 ping이 안 된다면 VMware 네트워크 설정(둘 다 같은 네트워크 모드: Bridge 또는 Host-only)으로 바꿔야 함.
> 

---

# 2) MySQL 컨테이너를 포트 노출하여 실행 (예시)

아직 도커 컨테이너를 포트로 노출하지 않았다면 아래처럼 실행(또는 기존 컨테이너 중지 후 재실행):

```bash
docker run -d \
  --name mysql-test \
  -e MYSQL_ROOT_PASSWORD=RootPass123! \
  -e MYSQL_DATABASE=testdb \
  -p 3306:3306 \
  mysql:8 \
  --default-authentication-plugin=mysql_native_password

```

설명:

- `p 3306:3306` 으로 호스트(우분투)의 3306 포트를 컨테이너에 매핑 → 다른 VM에서 `호스트IP:3306`으로 접근 가능.
- `-default-authentication-plugin=mysql_native_password`는 오래된 클라이언트 호환 문제 방지용(필요 시).

이미 컨테이너가 있고 포트가 안 열려 있으면 기존 컨테이너를 멈추고 위처럼 `-p` 옵션으로 재실행해야 함.

---

# 3) DB 사용자 만들기 — bee-box에서만 접속 허용(권장)

보안상 `'%`(모든 호스트 허용) 대신 bee-box IP만 허용하자. Ubuntu에서 다음을 실행:

컨테이너 내부에서 mysql 접속:

```bash
docker exec -it mysql-test mysql -u root -p
# 비밀번호: RootPass123!

```

MySQL 내에서:

```sql
-- 예: bee-box IP가 192.168.56.102 라고 가정
CREATE DATABASE IF NOT EXISTS gmshopdb CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci;
CREATE USER IF NOT EXISTS 'gmuser'@'192.168.56.102' IDENTIFIED BY 'GmPass!234';
GRANT ALL PRIVILEGES ON gmshopdb.* TO 'gmuser'@'192.168.56.102';
FLUSH PRIVILEGES;

```

만약 bee-box IP가 바뀔 가능성이 있거나 테스트 환경이면 임시로:

```sql
CREATE USER 'gmuser'@'%' IDENTIFIED BY 'GmPass!234';
GRANT ALL PRIVILEGES ON gmshopdb.* TO 'gmuser'@'%';
FLUSH PRIVILEGES;

```

> 권장: 운영/공개 환경이면 반드시 특정 IP만 허용.
> 

---

# 4) 방화벽(호스트 우분투) 열기

Ubuntu에 `ufw`가 활성화되어 있으면 bee-box에서 접근하도록 허용:

```bash
# bee-box IP가 192.168.56.102 인 경우
sudo ufw allow from 192.168.56.102 to any port 3306 proto tcp

# 또는 모든 내부 네트워크 허용(테스트)
sudo ufw allow 3306/tcp

```

확인:

```bash
sudo ufw status

```

---

# 5) bee-box에서 MySQL 클라이언트로 접속 테스트

bee-box VM에 mysql-client(또는 mariadb-client)가 있으면 아래처럼 테스트:

```bash
# (없으면 설치) Debian/Ubuntu 계열이면
sudo apt update && sudo apt install -y mysql-client

# 접속
mysql -h 192.168.56.101 -P 3306 -u gmuser -p
# 비밀번호: GmPass!234

```

접속되면 SQL 쿼리로 확인:

```sql
USE gmshopdb;
SHOW TABLES;

```

---

# 6) 웹앱(gmshop 등) 설정 변경 예제

gmshop(혹은 bee-box 내 웹앱)의 DB 설정 파일을 찾아 아래 항목을 호스트 IP로 바꿔준다.

예: `config.php` 또는 `db.php` 같은 파일에

```php
DB_HOST = '192.168.56.101';   // Ubuntu VM IP (도커가 노출한 호스트 IP)
DB_PORT = 3306;
DB_DATABASE = 'gmshopdb';
DB_USERNAME = 'gmuser';
DB_PASSWORD = 'GmPass!234';

```

또는 `.env` 있는 경우:

```
DB_HOST=192.168.56.101
DB_PORT=3306
DB_DATABASE=gmshopdb
DB_USERNAME=gmuser
DB_PASSWORD=GmPass!234

```

설정 변경 후 웹서버(APache/Nginx)나 웹앱을 재시작하면 연결 확인 가능.

---

# 7) 트러블슈팅 체크리스트

- ping이 동작하는가? (네트워크 문제 확인)
- Docker 컨테이너가 `p 3306:3306`로 실행 중인가? (`docker ps`로 포트 매핑 확인)
- MySQL 사용자가 정확한 호스트(bee-box IP 또는 `%`)로 생성되었는가? (`SELECT user, host FROM mysql.user;`)
- ufw/iptables 등 방화벽이 포트 3306을 차단하고 있지 않은가?
- MySQL 로그(`/var/lib/docker/...` 혹은 컨테이너 내부 `/var/log/mysql` 등) 확인 — 인증 실패 메시지 참조.
- 클라이언트 쪽에서 `mysql -h <host> -P 3306 -u <user> -p` 로 직접 연결 테스트를 먼저 해보자.

---

# 8) 보안 권장사항 (테스트 후 꼭!)

- 절대 인터넷에 3306 포트를 직접 노출하지 마세요. (VM/네트워크 격리 유지)
- 운영 전엔 강한 비밀번호, 특정 IP 허용, DB 계정 권한 최소화 적용.
- 가능하면 SSH 터널링(또는 VPN)을 통해 안전하게 접근하도록 구성